# 🤖 Machine Learning Algorithms Summary

Este notebook resume los principales algoritmos de *machine learning supervisado*, tanto para clasificación como para regresión, aunque nos centraremos más en clasificación.

Usaremos el dataset del Titanic como ejemplo práctico para comparar:

✅ Cómo funcionan  
✅ Cuándo usarlos  
✅ Sus ventajas y desventajas  
✅ Qué hiperparámetros son importantes  
✅ Qué tan bien predicen si alguien sobrevivió



## Preparación del dataset

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cargar y preparar el dataset
df = sns.load_dataset('titanic')

# Preprocesamiento similar al notebook anterior
df = df.drop(columns=['deck', 'embark_town', 'alive'])
df['age'] = df['age'].fillna(df['age'].median())
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])
df['sex'] = df['sex'].map({'male': 0, 'female': 1})
df = pd.get_dummies(df, columns=['embarked'], drop_first=True)

# Crear nuevas variables
df['family_size'] = df['sibsp'] + df['parch'] + 1
df['is_alone'] = (df['family_size'] == 1).astype(int)

# Escalar
scaler = StandardScaler()
df[['age', 'fare']] = scaler.fit_transform(df[['age', 'fare']])

# Selección de variables
features = ['pclass', 'sex', 'age', 'fare', 'family_size', 'is_alone', 'embarked_Q', 'embarked_S']
X = df[features]
y = df['survived']

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Modelos de clasificación

Vamos a probar:
- Logistic Regression
- K-Nearest Neighbors (KNN)
- Decision Trees
- Random Forest
- Support Vector Machines (SVM)
- Gradient Boosting (opcional)

**A. Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

Calcula una combinación lineal de las variables para estimar la probabilidad de una clase (por ejemplo, sobrevivir = 1). La regresión logística te dice cuánto aumenta o disminuye la probabilidad de que ocurra algo al cambiar una variable.

**¿Cuándo usarla?**  
- Cuando las variables tienen relación lineal con la probabilidad de clase.
- Es rápida, interpretable y sirve de baseline.

⚠️ **Limitaciones**: No modela relaciones no lineales.

Coeficientes importantes: Las que tengan coeficientes (model.coef_) más alejados de 0. Por ejemplo, sex, pclass, fare.


**B. K-Nearest Neighbors (KNN)**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

Mide la "distancia" entre puntos para clasificarlos (por ejemplo, si tus 5 vecinos más cercanos sobrevivieron, tú también probablemente). No es interpretable directamente, pero funciona bien cuando los grupos están claramente separados en el espacio.

**¿Cuándo usarlo?**  
- Cuando no hay mucha data y el espacio de características tiene significado geométrico.
- No necesita entrenamiento (lazy learner), solo compara con vecinos.

⚠️ **Limitaciones**: Sensible a escala y a ruido, lento con grandes datasets.

Hiperparámetro: `n_neighbors`, `metric`. No hay pesos directos, pero las que afecten más la distancia son las que más influyen. Por eso se deben escalar los datos.


**C. Árboles de decisión**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=4, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

Dividen el dataset en ramas según condiciones en las variables. Son muy explicativos: puedes ver qué reglas tomó el árbol para llegar a una predicción.

**¿Cuándo usarlo?**  
- Si quieres interpretabilidad y reglas claras.
- Puede capturar relaciones no lineales fácilmente.

⚠️ **Limitaciones**: Overfitting si no se poda (`max_depth`, `min_samples_leaf`)

Variables importantes: Las que más se usan para dividir los datos y reducir incertidumbre (model.feature_importances_).


**D. Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

Combina el resultado de varios árboles de decisión para obtener una solución.

**¿Cuándo usarlo?**  
- Cuando quieres precisión y evitar overfitting.
- Muy bueno para variables mixtas (numéricas + categóricas).

⚠️ **Limitaciones**: Menos interpretable, más pesado.

Hiperparámetros: `n_estimators`, `max_depth`


**E. SVM (Support Vector Machine)**

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='rbf', C=1, gamma='scale')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

Encuentra un hiperplano (una frontera) que separa clases lo mejor posible. Más complejo de interpretar, pero útil para datos donde las clases no se separan fácilmente.

**¿Cuándo usarla?**  
- Muy buena en datasets con muchas dimensiones y márgenes claros.
- Potente con pocas observaciones y datos escalados.

⚠️ **Limitaciones**: Costosa en grandes datasets, menos interpretable.

Hiperparámetros: `C`, `kernel`, `gamma`. Es necesario definir las variables que determinan la posición del margen del hp.

**F. Gradient Boosting**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

Crea árboles secuenciales, donde cada nuevo árbol corrige errores del anterior.Es potente y flexible, pero no tan fácil de explicar sin herramientas específicas.

**¿Cuándo usarlo?**  
- Cuando quieres el mejor rendimiento posible (competencias Kaggle).
- Captura relaciones complejas, robusto a ruido.

⚠️ **Limitaciones**: Requiere tuning de muchos hiperparámetros, más lento de entrenar.

Variables importantes: De nuevo `.feature_importances_`. También puedes usar técnicas como SHAP para explicabilidad más profunda.

## Comparativa de resultados

In [ ]:
from sklearn.metrics import accuracy_score

models = {
    'Logistic Regression': LogisticRegression(),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'Gradient Boosting': GradientBoostingClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    acc = accuracy_score(y_test, pred)
    print(f"{name}: {acc:.3f}")

## Conclusiones

| Modelo               | Pros                           | Contras                          |
|----------------------|--------------------------------|----------------------------------|
| Logistic Regression  | Simple, interpretable          | No capta no-linealidad          |
| KNN                  | Fácil, no entrena              | Lento, sensible a escala         |
| Árbol de decisión    | Interpretables, no-linealidad  | Overfitting sin poda             |
| Random Forest        | Robusto, preciso                | Menos interpretable              |
| SVM                  | Preciso en alta dimensión       | Lento, menos interpretable       |
| Gradient Boosting    | Muy preciso                     | Más costoso, tuning complejo     |

El mejor modelo depende del problema, pero entender cómo funciona cada uno es el primer paso hacia una buena solución. 🎯


## (ANEXO) GLM: Generalized Linear Models
Los GLM (Modelos Lineales Generalizados) son una generalización de la regresión lineal/logística que permiten otros tipos de distribución de error y funciones de enlace.

In [ ]:
import statsmodels.api as sm

X_glm = sm.add_constant(X_train)  # se agrega constante (intercepto)
glm_model = sm.GLM(y_train, X_glm, family=sm.families.Binomial())
glm_results = glm_model.fit()

print(glm_results.summary())

El resumen contiene:
* Una tabla detallada con los coeficientes de cada variable
* Valores p para evaluar la significancia estadística
* Odds ratio interpretables
* R^2 pseudo para calidad del ajuste


**¿Cuándo usarla?**  
- Cuando necesitas interpretabilidad estadística fuerte
- Cuando el proyecto es más científico o académico
- Cuando quieres probar hipótesis sobre variables específicas

> Otros tipos de familias son: Poisson (para contar eventos (ej. número de compras)) o Gamma (para valores positivos continuos (costes, tiempos))